## MiniGPT --> Word level language model trained on tinyshakespeare dataset

In [1]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
# let's look at the first 1000 characters
print(text[:102])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You a


In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [6]:
import torch
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
from typing import List
from torch.utils.data import DataLoader, TensorDataset, random_split

def tokenize_sentences(sentences: List[str], vocab_size: int = 100000):
    """Tokenize sentences and create a vocabulary."""
    tokenized_sentences = [sentence.lower().split() for sentence in sentences]
    all_words = [word for sentence in tokenized_sentences for word in sentence]
    
    # Build a vocabulary of the most common words
    word_counts = Counter(all_words)
    vocab = {word: i+2 for i, (word, _) in enumerate(word_counts.most_common(vocab_size))}
    inv_vocab={i+2: word for i, (word, _) in enumerate(word_counts.most_common(vocab_size))}
    vocab['<pad>'] = 0
    vocab['<unk>'] = 1
    inv_vocab[0]='<pad>'
    inv_vocab[1]='<unk>'
    
    return tokenized_sentences, vocab ,inv_vocab

def sentence_to_tensor(sentence: List[str], vocab: dict, sequence_length: int = 10):
    """Convert a sentence to a tensor of fixed length."""
    indices = [vocab.get(word, vocab['<unk>']) for word in sentence[:sequence_length]]
    return torch.tensor(indices, dtype=torch.long)


def prepare_dataset(sentences: List[str], context_length: int, max_length: int, batch_size: int):
    """Prepare the dataset and split into train, validation, and test sets."""
    tokenized_sentences, vocab,inv_vocab = tokenize_sentences(sentences)
    input_tensors = []
    target_tensors = []

    for sentence in tokenized_sentences:
        if len(sentence)<=1:
            continue
        sentence = sentence[:max_length] + ['<pad>'] * max(0, max_length - len(sentence))
        for i in range(len(sentence) - context_length):
            input_seq = sentence[i:i+context_length]
            target_seq = sentence[i+1:i+1+context_length]
            input_tensors.append(sentence_to_tensor(input_seq, vocab, context_length))
            target_tensors.append(sentence_to_tensor(target_seq, vocab, context_length))

    input_tensor = torch.stack(input_tensors)
    target_tensor = torch.stack(target_tensors)
    # Create TensorDataset
    dataset = TensorDataset(input_tensor, target_tensor)
    
    # Split dataset into train, validation, and test sets
    train_size = int(0.7 * len(dataset))
    valid_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - valid_size
    train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

    # Create DataLoader for each set
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    #return train_loader, valid_loader, test_loader, vocab
    return train_loader,valid_loader,test_loader,vocab,inv_vocab

In [7]:
sentences=text.split('\n')
#input,target,vocab=prepare_dataset(sentences,context_length=15,max_length=50)

context_length=5
max_length=10
batch_size=3
train_loader, valid_loader, test_loader, vocab ,inv_vocab= prepare_dataset(sentences, context_length, max_length, batch_size)

In [8]:
print(f" train_loader length :{train_loader.__len__()}")

#print(f" target.shape :{target.shape}")

 train_loader length :31858


In [9]:
print(f"Length of sentences :{len(sentences)}")
vocab_size=len(vocab)
print(f"vocab size: {vocab_size}")

Length of sentences :40001
vocab size: 23643


In [22]:
# hyperparameters
maxlen=50
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 15 # what is the maximum context length for predictions?
max_iters = 10000
eval_interval = 100
learning_rate = 1e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 512
n_head = 4
n_layer = 4
dropout = 0.0

In [26]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class miniGPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        #print(idx.shape)
        B,T = idx.shape
        #print(f"B : {B} C:{T}")
        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

In [30]:
from tqdm import tqdm
num_epochs=10
model = miniGPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

n_embd = 512
n_head = 8
n_layer = 6
dropout = 0.1
learning_rate=4e-5
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
eval_interval=1000
print(f"learning_rate:{learning_rate} num_head: {n_head} number_of_embeddings: {n_embd} number_of_layers: {n_layer}")
for epoch in range(num_epochs):
    model.train()
    total_loss=0
    for iter, (xb, yb) in tqdm(enumerate(train_loader)):
        xb, yb = xb.to(device), yb.to(device)
        logits, loss = model(xb, yb)
        total_loss+=loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        if iter % eval_interval == 0:
            print(f"Epoch {epoch}, Step {iter}: train loss {loss:.4f}")
    avg_loss=total_loss/len(train_loader)
    print(f"==========Epoch {epoch}: Average training loss: {avg_loss:.4f}=============")
    
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for xv, yv in valid_loader:
            xv, yv = xv.to(device), yv.to(device)
            val_logits, val_loss = model(xv, yv)
            total_val_loss += val_loss.item()
    
    avg_val_loss = total_val_loss / len(valid_loader)
    print(f"Epoch {epoch}: Average validation loss: {avg_val_loss:.4f}")
    checkpoint = {
        "state_dict": model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch": epoch,
    }
    torch.save(checkpoint, f"checkpoint_epoch_{epoch}.pth.tar")



36.846171 M parameters
learning_rate:4e-05 num_head: 8 number_of_embeddings: 512 number_of_layers: 6


4it [00:00, 13.69it/s]

Epoch 0, Step 0: train loss 10.2910


1004it [00:47, 20.70it/s]

Epoch 0, Step 1000: train loss 6.8119


...

Epoch 9, Step 30000: train loss 3.2582


31004it [24:49, 20.85it/s]

Epoch 9, Step 31000: train loss 0.1406


31858it [25:30, 20.82it/s]


==========Epoch 9: Average training loss: 2.1681=============
Epoch 9: Average validation loss: 3.2572


In [53]:
# generate from the model
seed=[[vocab['he']]]
context = torch.tensor(seed, dtype=torch.long, device=device)
output_tokens=m.generate(context, max_new_tokens=200)[0].tolist()
res=''
for i in output_tokens:
  if inv_vocab[i]=='<pad>':
    continue
  res+=inv_vocab[i]
  res+=' '
print(res)

he may ours. your broken in before, sorrow lucentio. 
